In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7192716cd4c51e68d83bb907fc83f1ebe8c48ea77f40aec8e2da08e452f0d4da
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import os
from PIL import Image
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import pickle
from sklearn import manifold
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report
import wandb
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
zip_path = "/content/gdrive/MyDrive/svhn_yolo.zip"

Mounted at /content/gdrive


In [ ]:
import zipfile

# Open the .zip file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    # Extract all the contents of the .zip file
    zip_ref.extractall("SVHN_YOLO")

In [ ]:
import os

# Define the main folder path
main_folder_path = "svhn_folder"

# Define the subfolder paths
images_train_path = os.path.join(main_folder_path, "images", "train")
images_val_path = os.path.join(main_folder_path, "images", "val")
labels_train_path = os.path.join(main_folder_path, "labels", "train")
labels_val_path = os.path.join(main_folder_path, "labels", "val")

# Create the directory structure
os.makedirs(images_train_path, exist_ok=True)
os.makedirs(images_val_path, exist_ok=True)
os.makedirs(labels_train_path, exist_ok=True)
os.makedirs(labels_val_path, exist_ok=True)


In [ ]:
!apt-get update
!apt-get install tree


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [ ]:
!tree svhn_folder

svhn_folder
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories, 0 files


In [ ]:
import glob

# Define the folder path
images_path = "SVHN_YOLO/svhn/images/all"
labels_path = "SVHN_YOLO/svhn/labels/all"

# Get the names of all the ".png" files in the folder
image_files = glob.glob(os.path.join(images_path, "*.png"))
label_files = glob.glob(os.path.join(labels_path, "*.txt"))

# Extract the file names from the file paths
images_list = [os.path.basename(file_path) for file_path in image_files]
labels_list = [os.path.basename(file_path) for file_path in label_files]

print(len(images_list))
print(len(labels_list))


33402
33402


In [ ]:
num_samples = len(images_list)

In [ ]:
# Split the data into training, validation, and testing sets
val_split = int(0.2 * num_samples)
test_split = int(0.1 * num_samples) + val_split

In [ ]:
# validation data splitted to 20% of the dataset
images_val_data = images_list[:val_split]  
labels_val_data = labels_list[:val_split]

# testing data splitted to 10% of the dataset
images_test_data = images_list[val_split:test_split]
labels_test_data = labels_list[val_split:test_split]

# rest reserved for training data
images_train_data = images_list[test_split:]
labels_train_data = labels_list[test_split:]

In [ ]:
import shutil

src_folder1 = "SVHN_YOLO/svhn/images/all"
dst_folder1 = "svhn_folder/images/train"

for image_name in images_train_data:
    src_file = os.path.join(src_folder1, image_name)
    dst_file = os.path.join(dst_folder1, image_name)
    shutil.copy(src_file, dst_file)

src_folder2 = "SVHN_YOLO/svhn/images/all"
dst_folder2 = "svhn_folder/images/val"

for image_name in images_val_data:
    src_file = os.path.join(src_folder2, image_name)
    dst_file = os.path.join(dst_folder2, image_name)
    shutil.copy(src_file, dst_file)

src_folder3 = "SVHN_YOLO/svhn/labels/all"
dst_folder3 = "svhn_folder/labels/train"

for image_name in labels_train_data:
    src_file = os.path.join(src_folder3, image_name)
    dst_file = os.path.join(dst_folder3, image_name)
    shutil.copy(src_file, dst_file)


src_folder4 = "SVHN_YOLO/svhn/labels/all"
dst_folder4 = "svhn_folder/labels/val"

for image_name in labels_val_data:
    src_file = os.path.join(src_folder4, image_name)
    dst_file = os.path.join(dst_folder4, image_name)
    shutil.copy(src_file, dst_file)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SVHN_YOLO_Dataset(Dataset):
    def __init__(self, img_file, lab_file, transform=None):
        self.image_files = img_file
        self.label_files = lab_file

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self):
        return self.image_files, self.label_files

train_set = SVHN_YOLO_Dataset(images_train_data, labels_train_data)
test_set = SVHN_YOLO_Dataset(images_test_data, labels_test_data)
val_set = SVHN_YOLO_Dataset(images_val_data, labels_val_data)

# Define the data loaders for each split
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_set, batch_size=100, shuffle=True)

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15232, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15232 (delta 5), reused 10 (delta 4), pack-reused 15218
Receiving objects: 100% (15232/15232), 14.17 MiB | 32.10 MiB/s, done.
Resolving deltas: 100% (10441/10441), done.
/content/yolov5


In [ ]:
!python train.py --img 256 --cfg yolov5x.yaml --batch 10 --epochs 10 --data svhn.yaml --weights yolov5x.pt --workers 24 --name cv_q3_yolo

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=svhn.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=10, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=cv_q3_yolo, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://gith

# Techniques used for Data Augmentation are Random Cropping, Random Erasing and Random Rotation.


### Random Cropping

In [ ]:
def random_crop(img, sz):
    # height, width = img.shape[:2]
    h = img.shape[0]
    w = img.shape[1]
    ch, cw = sz

    y = np.random.randint(0, h - ch + 1)
    x = np.random.randint(0, w - cw + 1)

    return img[y:y+ch, x:x+cw, :]


## Random Erasing


In [ ]:
def random_erasing(img):

    if np.random.uniform() > 0.5:
        return img

    h = img.shape[0]
    w = img.shape[1]
    c = img.shape[2]

    # h, w, c = img.shape
    area = h * w

    while True:
        target_area = np.random.uniform(0.02, 0.4) * area
        aspect_ratio = np.random.uniform(0.3, 3.3)
        er_h = int(round(np.sqrt(target_area * aspect_ratio)))
        er_w = int(round(np.sqrt(target_area / aspect_ratio)))
        if er_h < h and er_w < w:
            break

    x1 = np.random.randint(0, h - er_h)
    y1 = np.random.randint(0, w - er_w)

    if c == 1:
        img[x1:x1+er_h, y1:y1+er_w] = np.random.uniform(0, 1)
    else:
        for i in range(c):
            img[x1:x1+er_h, y1:y1+er_w, i] = np.random.uniform(0, 1)

    return img


## Random Rotation

In [ ]:
def random_rotation(img, angle_range=(-15, 15)):

    # Generating a random rotation
    angle = np.random.uniform(angle_range[0], angle_range[1])

    # Image center
    h, w = img.shape[:2]
    cx, cy = w // 2, h // 2

    # Rotation matrix
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)

    # Apply the rotation 
    rotated_img = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    return rotated_img
